### Installation

In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab and Kaggle notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
    !pip install --no-deps cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True


fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.15 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the `Llama-3.1` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. But we convert it to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`/ Llama-3 renders multi turn conversations like below:

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3` and more.

In [5]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("mlabonne/FineTome-100k", split = "train")

README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

We now use `standardize_sharegpt` to convert ShareGPT style datasets into HuggingFace's generic format. This changes the dataset from looking like:
```
{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
```
to
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

In [8]:
import random
from faker import Faker
import json

# Initialize the Faker object
fake = Faker()

# Sample names for landlords and tenants
landlord_names = ["John Doe", "Jane Doe", "Mary Johnson", "Michael Brown", "Sarah Davis"]
tenant_names = ["Emily White", "James Black", "Sophia Green", "Daniel Adams", "Olivia Taylor"]

# Function to generate random California addresses
def generate_california_address():
    # Generate a random California address (e.g., address in San Francisco)
    address = fake.address().replace('\n', ', ')  # Ensure the address is one-line
    city =   "CA"
    zip_code = random.choice([str(random.randint(90001, 96162))])  # California zip code range
    return f"{address}, {city} {zip_code}"

# Function to generate a Rent Agreement example in the conversation format
def generate_rent_agreement_conversation():
    # Select random landlord, tenant, and other details
    landlord = random.choice(landlord_names)
    tenant = random.choice(tenant_names)
    address = generate_california_address()
    lease_term = random.choice(['6 months', '12 months', '24 months'])
    monthly_rent = random.randint(1000, 5000)
    security_deposit = monthly_rent  # Typically equal to rent

    # Generate human question asking for the rent agreement
    human_message = f"Generate a Rent Agreement contract using these details: Landlord: {landlord}; Tenant: {tenant}; Address: {address}; Lease Term: {lease_term}; Monthly Rent: ${monthly_rent}; Security Deposit: ${security_deposit}."

    # Generate GPT response (this would be a sample of what the rent agreement might look like)
    gpt_message = f"RENT AGREEMENT\n\nThis Rent Agreement is made effective as of [Date] between {landlord} (\"Landlord\") and {tenant} (\"Tenant\") for the property located at {address}. The Lease Term is {lease_term}, with a Monthly Rent of ${monthly_rent} due on the first day of each month. A Security Deposit of ${security_deposit} is due upon signing this agreement. Tenant agrees to maintain the property and abide by all terms outlined herein."

    return {
        "conversations": [
            {"from": "human", "value": human_message},
            {"from": "gpt", "value": gpt_message}
        ],
        "source": "WebInstructSub_axolotl",
        "score": random.uniform(4.0, 5.5)  # Random score for each conversation
    }

# Generate 100 dataset samples
def generate_rent_agreement_dataset(num_samples):
    dataset = []

    for _ in range(num_samples):
        dataset.append(generate_rent_agreement_conversation())

    return dataset

# Generate 100 dataset samples
dataset = generate_rent_agreement_dataset(100)

# Save the dataset to a file
with open('california_rent_agreements_conversations.json', 'w') as f:
    json.dump(dataset, f, indent=4)

print("Dataset generated and saved as 'california_rent_agreements_conversations.json'.")


Dataset generated and saved as 'california_rent_agreements_conversations.json'.


In [9]:
# prompt: california_rent_agreements_conversations.json to dataset

import json
from datasets import Dataset

# Load the JSON data
with open('california_rent_agreements_conversations.json', 'r') as f:
    data = json.load(f)

# Convert the list of dictionaries to a Dataset object
dataset = Dataset.from_list(data)

# Now you can work with the 'dataset' object
dataset


Dataset({
    features: ['conversations', 'source', 'score'],
    num_rows: 100
})

In [10]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Standardizing format:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

We look at how the conversations are structured for item 5:

In [11]:
dataset[5]["conversations"]

[{'content': 'Generate a Rent Agreement contract using these details: Landlord: Mary Johnson; Tenant: Olivia Taylor; Address: 2377 Lawrence Meadow, Beckyton, TX 86795, CA 91120; Lease Term: 12 months; Monthly Rent: $3426; Security Deposit: $3426.',
  'role': 'user'},
 {'content': 'RENT AGREEMENT\n\nThis Rent Agreement is made effective as of [Date] between Mary Johnson ("Landlord") and Olivia Taylor ("Tenant") for the property located at 2377 Lawrence Meadow, Beckyton, TX 86795, CA 91120. The Lease Term is 12 months, with a Monthly Rent of $3426 due on the first day of each month. A Security Deposit of $3426 is due upon signing this agreement. Tenant agrees to maintain the property and abide by all terms outlined herein.',
  'role': 'assistant'}]

And we see how the chat template transformed these conversations.

**[Notice]** Llama 3.1 Instruct's default chat template default adds `"Cutting Knowledge Date: December 2023\nToday Date: 26 July 2024"`, so do not be alarmed!

In [12]:
dataset[5]

{'conversations': [{'content': 'Generate a Rent Agreement contract using these details: Landlord: Mary Johnson; Tenant: Olivia Taylor; Address: 2377 Lawrence Meadow, Beckyton, TX 86795, CA 91120; Lease Term: 12 months; Monthly Rent: $3426; Security Deposit: $3426.',
   'role': 'user'},
  {'content': 'RENT AGREEMENT\n\nThis Rent Agreement is made effective as of [Date] between Mary Johnson ("Landlord") and Olivia Taylor ("Tenant") for the property located at 2377 Lawrence Meadow, Beckyton, TX 86795, CA 91120. The Lease Term is 12 months, with a Monthly Rent of $3426 due on the first day of each month. A Security Deposit of $3426 is due upon signing this agreement. Tenant agrees to maintain the property and abide by all terms outlined herein.',
   'role': 'assistant'}],
 'source': 'WebInstructSub_axolotl',
 'score': 5.09059144460829,
 'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|star

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [13]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Converting train dataset to ChatML (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [118]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

We verify masking is actually done:

In [119]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGenerate a Rent Agreement contract using these details: Landlord: Michael Brown; Tenant: Daniel Adams; Address: 849 Mallory Rapids Suite 028, Lake Ryanport, MO 38222, CA 90687; Lease Term: 24 months; Monthly Rent: $1640; Security Deposit: $1640.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nRENT AGREEMENT\n\nThis Rent Agreement is made effective as of [Date] between Michael Brown ("Landlord") and Daniel Adams ("Tenant") for the property located at 849 Mallory Rapids Suite 028, Lake Ryanport, MO 38222, CA 90687. The Lease Term is 24 months, with a Monthly Rent of $1640 due on the first day of each month. A Security Deposit of $1640 is due upon signing this agreement. Tenant agrees to maintain the property and abide by all terms outlined herein.<|eot_id|>'

In [120]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                                                     \n\nRENT AGREEMENT\n\nThis Rent Agreement is made effective as of [Date] between Michael Brown ("Landlord") and Daniel Adams ("Tenant") for the property located at 849 Mallory Rapids Suite 028, Lake Ryanport, MO 38222, CA 90687. The Lease Term is 24 months, with a Monthly Rent of $1640 due on the first day of each month. A Security Deposit of $1640 is due upon signing this agreement. Tenant agrees to maintain the property and abide by all terms outlined herein.<|eot_id|>'

We can see the System and Instruction prompts are successfully masked!

In [121]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
8.039 GB of memory reserved.


In [122]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 100 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,1.202700
2,1.212800
3,1.154200
4,0.950300
5,0.695700
6,0.468200
7,0.315600
8,0.193800
9,0.080800
10,0.032000


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

446.5262 seconds used for training.
7.44 minutes used for training.
Peak reserved memory = 6.531 GB.
Peak reserved memory for training = 3.896 GB.
Peak reserved memory % of max memory = 44.284 %.
Peak reserved memory for training % of max memory = 26.417 %.


 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [132]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [133]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

In [1]:
# prompt: run a prompt against trained model

prompt = "Generate a Rent Agreement contract using these details: Landlord: Vidit Smith; Tenant: Alice Johnson; Address: 123 Main St, Anytown, CA 91234; Lease Term: 12 months; Monthly Rent: $2500; Security Deposit: $2500."

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=512)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


NameError: name 'tokenizer' is not defined

In [136]:
dataset[10]

{'conversations': [{'content': 'Generate a Rent Agreement contract using these details: Landlord: Sarah Davis; Tenant: James Black; Address: 6460 King Locks Apt. 935, New Robert, WY 20744, CA 92184; Lease Term: 12 months; Monthly Rent: $4021; Security Deposit: $4021.',
   'role': 'user'},
  {'content': 'RENT AGREEMENT\n\nThis Rent Agreement is made effective as of [Date] between Sarah Davis ("Landlord") and James Black ("Tenant") for the property located at 6460 King Locks Apt. 935, New Robert, WY 20744, CA 92184. The Lease Term is 12 months, with a Monthly Rent of $4021 due on the first day of each month. A Security Deposit of $4021 is due upon signing this agreement. Tenant agrees to maintain the property and abide by all terms outlined herein.',
   'role': 'assistant'}],
 'source': 'WebInstructSub_axolotl',
 'score': 4.332155747045972}